# Comparison of velocyto vs. Alevin-fry

In [15]:
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv
import matplotlib as mpl
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity
import gzip
import pickle

## Load data

In [2]:
# Load AnnData objects
adatas = {
    dataset: {
        method: sc.read_h5ad(
            f"../{dataset.capitalize()}/data/{dataset}_{method}_sto.h5ad"
        ) for method in ["velocyto", "alevin"]
    } for dataset in ["stewart", "mathew", "fu"]
}

In [3]:
adatas["fu"]["velocyto"].obs["cluster"] = adatas["fu"]["velocyto"].obs["cluster"].astype(str).replace("nan", "Others").astype("category")
adatas["fu"]["alevin"].obs["cluster"] = adatas["fu"]["alevin"].obs["cluster"].astype(str).replace("nan", "Others").astype("category")

In [4]:
# Separate Fu into paired-end and single-end
fu_se_samples = ["MJ001", "MJ002", "MJ003", "MJ017", "MJ016"]
adatas["fu_se"] = {
    tool: adata[adata.obs["id"].isin(fu_se_samples), :]
    for tool, adata in adatas["fu"].items()
}
adatas["fu_pe"] = {
    tool: adata[~adata.obs["id"].isin(fu_se_samples), :]
    for tool, adata in adatas["fu"].items()
}

In [5]:
# Check that the velocyto and alevin-fry datasets have the same cells/genes
for dataset, tool_dict in adatas.items():
    assert np.all(
        tool_dict["velocyto"].obs_names == tool_dict["alevin"].obs_names
    ), f"Different cells for velocyto and alevin on {dataset}"
    assert np.all(
        tool_dict["velocyto"].var_names == tool_dict["alevin"].var_names
    ), f"Different genes for velocyto and alevin on {dataset}"

In [6]:
# Mappings for plot labelling
method_dict = {
    "velocyto": "Velocyto",
    "alevin": "Alevin-fry",
    "cellranger": "Cell Ranger"
}
dataset_dict = {
    "stewart": "Stewart et al.",
    "mathew": "Mathew et al.",
    "fu": "Fu et al.",
    "fu_se": "Fu et al. (SE)",
    "fu_pe": "Fu et al. (PE)"
}

## Figure 1: Compare counts

In [7]:
# Calculate total counts per cell
total_counts_df = pd.concat(
    [
        pd.DataFrame(
            data={
                tool: np.asarray(
                    adata.layers["spliced_counts"].sum(axis=1) +
                    adata.layers["unspliced_counts"].sum(axis=1) +
                    adata.layers["ambiguous_counts"].sum(axis=1)
                ).ravel() for tool, adata in sub_dict.items()
            }
        ).assign(
            dataset=dataset_name,
            cellranger=np.asarray(
                sub_dict["velocyto"].layers["counts"].sum(axis=1)
            ).ravel()
        )
        for dataset_name, sub_dict in adatas.items() if dataset_name != "fu"
    ]
)


In [8]:
# Calculate count difference between velocyto/alevin-fry with cellranger
total_counts_diff = total_counts_df[["velocyto", "alevin", "dataset"]].copy()
total_counts_diff["velocyto"] = total_counts_df["cellranger"] - total_counts_diff["velocyto"]
total_counts_diff["alevin"] = total_counts_df["cellranger"] - total_counts_diff["alevin"]

total_counts_df = total_counts_df.melt(
    id_vars="dataset", var_name="method", value_name="counts"
)
total_counts_diff= total_counts_diff.melt(
    id_vars="dataset", var_name="method", value_name="difference"
)

In [9]:
# Correlations of spliced counts with CellRanger counts of same gene
spliced_total_corrs = pd.concat(
    [
        pd.DataFrame(
            data={
                "velocyto": stats.pearsonr(
                    sub_dict["velocyto"].layers["counts"].toarray(),
                    sub_dict["velocyto"].layers["spliced_counts"].toarray(),
                    axis=0
                )[0],
                "alevin": stats.pearsonr(
                    sub_dict["velocyto"].layers["counts"].toarray(),
                    sub_dict["alevin"].layers["spliced_counts"].toarray(),
                    axis=0
                )[0],
                "dataset": dataset
            }
        )
        for dataset, sub_dict in adatas.items() if dataset != "fu"
    ]
).melt(
    id_vars="dataset", var_name="method", value_name="pearsonr"
)

In [10]:
# Calculate ratio of unspliced counts per cell
unspliced_ratios = pd.concat(
    [
        pd.DataFrame(
            data={
                tool: np.asarray(
                    adata.layers["unspliced_counts"].sum(axis=1) /
                    (
                        adata.layers["spliced_counts"].sum(axis=1) +
                        adata.layers["unspliced_counts"].sum(axis=1) +
                        adata.layers["ambiguous_counts"].sum(axis=1)
                    )
                ).ravel() * 100 for tool, adata in sub_dict.items()
            }
        ).assign(dataset=dataset)
        for dataset, sub_dict in adatas.items() if dataset != "fu"
    ]
).melt(
    id_vars="dataset", var_name="method", value_name="unspliced_ratio"
)

In [11]:
# save
total_counts_df.to_parquet("data/1_total_counts.parquet.gzip", compression="gzip", index=True)
total_counts_diff.to_parquet("data/1_total_counts_diff.parquet.gzip", compression="gzip", index=True)
spliced_total_corrs.to_parquet("data/1_spliced_total_corrs.parquet.gzip", compression="gzip", index=True)
unspliced_ratios.to_parquet("data/1_unspliced_ratios.parquet.gzip", compression="gzip", index=True)

## Figure 2: Correlations of overlapping genes

In [12]:
# Get overlapping genes
overlapping_genes = {
    "stewart": pd.read_pickle(
        "../Stewart/data/stewart_overlapping_genes.pkl.gz",
        compression="gzip"
    ),
    "mathew": pd.read_pickle(
        "../Mathew/data/mathew_overlapping_genes.pkl.gz",
        compression='gzip'
    ),
    "fu": pd.read_pickle(
        "../Fu/data/fu_overlapping_genes.pkl.gz",
        compression='gzip'
    )
}

# Subset to only genes with mean expression >= 1
overlapping_genes = {
    dataset: df[df["mean counts cellranger"] >= 1].sort_values(
        by=["rel_overlap", "overlap_length"], ascending=False
    ) for  dataset, df in overlapping_genes.items()
}

overlapping_genes["fu_pe"] = overlapping_genes["fu"]
overlapping_genes["fu_se"] = overlapping_genes["fu"]

In [13]:
# Get Pearson correlations
corrs_overlapping = pd.concat(
    [
        # <= 50% overlap
        pd.DataFrame(
            data={
                "velocyto": stats.pearsonr(
                    sub_dict["velocyto"][:, overlapping_genes[dataset][
                        overlapping_genes[dataset].rel_overlap <= 0.5
                    ].index.get_level_values(0)].layers["counts"].toarray(),
                    sub_dict["velocyto"][:, overlapping_genes[dataset][
                        overlapping_genes[dataset].rel_overlap <= 0.5
                    ].index.get_level_values(1)].layers["unspliced_counts"].toarray(),
                    axis=0
                )[0],
                "alevin": stats.pearsonr(
                    sub_dict["velocyto"][:, overlapping_genes[dataset][
                        overlapping_genes[dataset].rel_overlap <= 0.5
                    ].index.get_level_values(0)].layers["counts"].toarray(),
                    sub_dict["alevin"][:, overlapping_genes[dataset][
                        overlapping_genes[dataset].rel_overlap <= 0.5
                    ].index.get_level_values(1)].layers["unspliced_counts"].toarray(),
                    axis=0
                )[0],
                "dataset": dataset,
                "overlap": "overlap $\leq50\%$"
            }
        )
        for dataset, sub_dict in adatas.items() if dataset != "fu"
    ] + [
        # > 50%, <= 75% overlap
        pd.DataFrame(
            data={
                "velocyto": stats.pearsonr(
                    sub_dict["velocyto"][:, overlapping_genes[dataset][
                        (overlapping_genes[dataset].rel_overlap > 0.5) &
                        (overlapping_genes[dataset].rel_overlap <= 0.75)
                    ].index.get_level_values(0)].layers["counts"].toarray(),
                    sub_dict["velocyto"][:, overlapping_genes[dataset][
                        (overlapping_genes[dataset].rel_overlap > 0.5) &
                        (overlapping_genes[dataset].rel_overlap <= 0.75)
                    ].index.get_level_values(1)].layers["unspliced_counts"].toarray(),
                    axis=0
                )[0],
                "alevin": stats.pearsonr(
                    sub_dict["velocyto"][:, overlapping_genes[dataset][
                        (overlapping_genes[dataset].rel_overlap > 0.5) &
                        (overlapping_genes[dataset].rel_overlap <= 0.75)
                    ].index.get_level_values(0)].layers["counts"].toarray(),
                    sub_dict["alevin"][:, overlapping_genes[dataset][
                        (overlapping_genes[dataset].rel_overlap > 0.5) &
                        (overlapping_genes[dataset].rel_overlap <= 0.75)
                    ].index.get_level_values(1)].layers["unspliced_counts"].toarray(),
                    axis=0
                )[0],
                "dataset": dataset,
                "overlap": "$50\%<$ overlap $\leq75\%$"
            }
        )
        for dataset, sub_dict in adatas.items() if dataset != "fu"
    ] + [
        # > 75% overlap
        pd.DataFrame(
            data={
                "velocyto": stats.pearsonr(
                    sub_dict["velocyto"][:, overlapping_genes[dataset][
                        overlapping_genes[dataset].rel_overlap > 0.75
                    ].index.get_level_values(0)].layers["counts"].toarray(),
                    sub_dict["velocyto"][:, overlapping_genes[dataset][
                        overlapping_genes[dataset].rel_overlap > 0.75
                    ].index.get_level_values(1)].layers["unspliced_counts"].toarray(),
                    axis=0
                )[0],
                "alevin": stats.pearsonr(
                    sub_dict["velocyto"][:, overlapping_genes[dataset][
                        overlapping_genes[dataset].rel_overlap > 0.75
                    ].index.get_level_values(0)].layers["counts"].toarray(),
                    sub_dict["alevin"][:, overlapping_genes[dataset][
                        overlapping_genes[dataset].rel_overlap > 0.75
                    ].index.get_level_values(1)].layers["unspliced_counts"].toarray(),
                    axis=0
                )[0],
                "dataset": dataset,
                "overlap": "> 75%"
            }
        )
        for dataset, sub_dict in adatas.items() if dataset != "fu"
    ]
).melt(
    id_vars=["dataset", "overlap"], var_name="method", value_name="pearsonr"
)

In [16]:
# save
with gzip.open("data/2_overlapping_genes.gz", "wb") as f:
    pickle.dump(overlapping_genes, f)

corrs_overlapping.to_parquet("data/2_corrs_overlapping.parquet.gzip", compression="gzip", index=True)

## Figure 3: Velocity genes

In [17]:
# Get velocity genes overlaps
velocity_genes_df = pd.DataFrame(
    data={
        "velocyto": [
            np.sum(
                sub_dict["velocyto"].var["velocity_genes"] &
                ~sub_dict["alevin"].var["velocity_genes"]
                                     
            )
            for dataset, sub_dict in adatas.items()
        ],
        "alevin": [
            np.sum(
                ~sub_dict["velocyto"].var["velocity_genes"] &
                sub_dict["alevin"].var["velocity_genes"]
                                     
            )
            for dataset, sub_dict in adatas.items()
        ],
        "overlap": [
            np.sum(
                sub_dict["velocyto"].var["velocity_genes"] &
                sub_dict["alevin"].var["velocity_genes"]
                                     
            )
            for dataset, sub_dict in adatas.items()
        ]
    },
    index = [dataset_dict[key] for key in adatas.keys()]
)

In [18]:
# save
velocity_genes_df.to_parquet("data/3_velocity_genes.parquet.gz", compression="gzip", index=True)

## Figure 4: Comparison of velocities

In [19]:
# Compute cosine similarities of velocities per cell
cosine_sims = pd.concat(
    [
        pd.DataFrame(
            {
                "cosine_similarity": np.diagonal(
                    cosine_similarity(
                        sub_dict["velocyto"].layers["velocity"], sub_dict["alevin"].layers["velocity"]
                    )
                ),
                "cluster": sub_dict["velocyto"].obs["cluster"].copy(),
                "dataset": dataset
            }
        )
        for dataset, sub_dict in adatas.items() if not dataset.startswith("fu_")
    ]
)

In [20]:
# Compute pearson correlation of velocities per gene
velocity_corrs = pd.concat(
    [
        pd.DataFrame(
            data={
                "pearsonr": stats.pearsonr(
                    sub_dict["velocyto"].layers["velocity"],
                    sub_dict["alevin"].layers["velocity"],
                    axis=0
                )[0],
                "highly_variable": sub_dict["velocyto"].var["highly_variable"].copy(),
                "dataset": dataset
            }
        )
        for dataset, sub_dict in adatas.items() if not dataset.startswith("fu_")
    ]
).reset_index(drop=True)

In [21]:
# Get transition probability matrices
tpms = {
    dataset: {
        method: scv.utils.get_transition_matrix(adata)
        for method, adata in sub_dict.items()
    } for dataset, sub_dict in adatas.items() if not dataset.startswith("fu_")
}

In [22]:
# Compute Wasserstein distance of transition probability matrix
vals = {
    dataset: np.arange(tpm["velocyto"].shape[0])
    for dataset, tpm in tpms.items()
}
wasserstein_dists = pd.concat(
    [
        pd.DataFrame(
            {
                "wasserstein_distance": [
                    stats.wasserstein_distance(
                        vals[dataset], vals[dataset], row_vc.toarray()[0], row_af.toarray()[0]
                    )
                    for row_vc, row_af in zip(tpms[dataset]["velocyto"], tpms[dataset]["alevin"])
                ],
                "cluster": adatas[dataset]["velocyto"].obs["cluster"],
                "dataset": dataset
            }
        )
        for dataset, sub_dict in tpms.items()
    ]
)

In [23]:
# save
cosine_sims.to_parquet("data/4_cosine_sims.parquet.gzip", compression="gzip", index=True)
velocity_corrs.to_parquet("data/4_velocity_corrs.parquet.gzip", compression="gzip", index=True)
wasserstein_dists.to_parquet("data/4_wasserstein_dists.parquet.gzip", compression="gzip", index=True)

## Supplementary Figure 7: Are differences in velocities significant?

In [24]:
# Get sign changes in velocities
sign_changes = pd.concat(
    [
        pd.DataFrame(
            {
                "proportion": (
                    np.sign(sub_dict["velocyto"].layers["velocity"]) != np.sign(sub_dict["alevin"].layers["velocity"])
                ).sum(axis=1) / sub_dict["velocyto"].shape[1],
                "dataset": dataset
            }
        )
        for dataset, sub_dict in adatas.items() if not dataset.startswith("fu_")
    ]
)

In [25]:
# Get cosine similarities of velocities between neighboring cells
cosine_sims_neighbors = pd.concat(
    [
        pd.DataFrame(
            {
                tool: np.diagonal(
                    cosine_similarity(
                        adata.layers["velocity"],
                        adata.layers["velocity"][
                            np.argmin(
                                np.ma.masked_where(
                                    adata.obsp["distances"].toarray() == 0,
                                    adata.obsp["distances"].toarray()
                                    ),
                                axis=1
                            )
                        ]
                    )
                ) for tool, adata in sub_dict.items()
            } 
        ).assign(dataset=dataset)
        for dataset, sub_dict in adatas.items() if not dataset.startswith("fu_")
    ]
).melt(
    id_vars="dataset", var_name="method", value_name="cosine_similarity"
)

In [28]:
# Get TPMs with union of velocyto and alevin-fry velocity genes
tpms_union = {}
for dataset, sub_dict in adatas.items():
    if dataset.startswith("fu_"):
        continue
    tpms_union[dataset] = {}
    velocity_genes = np.logical_or(
        sub_dict["velocyto"].var["velocity_genes"], sub_dict["alevin"].var["velocity_genes"]
    )
    for tool, adata in sub_dict.items():
        adata_ = adata.copy()
        adata_.var["velocity_genes"] = velocity_genes
        scv.tl.velocity_graph(adata_, n_jobs=30)
        tpms_union[dataset][tool] = scv.utils.get_transition_matrix(adata_)
        del adata_

computing velocity graph (using 30/52 cores)


  0%|          | 0/7257 [00:00<?, ?cells/s]

    finished (0:00:42) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity graph (using 30/52 cores)


  0%|          | 0/7257 [00:00<?, ?cells/s]

    finished (0:00:08) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity graph (using 30/52 cores)


...

    finished (0:00:28) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity graph (using 30/52 cores)


  0%|          | 0/50033 [00:00<?, ?cells/s]

    finished (0:00:49) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity graph (using 30/52 cores)


  0%|          | 0/50033 [00:00<?, ?cells/s]

    finished (0:00:49) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [30]:
# Compute Wasserstein distance of transition probability matrix with union of velocity genes
vals = {
    dataset: np.arange(tpm["velocyto"].shape[0])
    for dataset, tpm in tpms_union.items()
}
wasserstein_dists_union = pd.concat(
    [
        pd.DataFrame(
            {
                "wasserstein_distance": [
                    stats.wasserstein_distance(
                        vals[dataset], vals[dataset], row_vc.toarray()[0], row_af.toarray()[0]
                    )
                    for row_vc, row_af in zip(sub_dict["velocyto"], sub_dict["alevin"])
                ],
                "cluster": adatas[dataset]["velocyto"].obs["cluster"],
                "dataset": dataset
            }
        )
        for dataset, sub_dict in tpms_union.items()
    ]
)

In [31]:
# save
sign_changes.to_parquet("data/s7_sign_changes.parquet.gzip", compression="gzip", index=True)
cosine_sims_neighbors.to_parquet("data/s7_cosine_sims_neighbors.parquet.gzip", compression="gzip", index=True)
wasserstein_dists_union.to_parquet("data/s7_wasserstein_dists_union.parquet.gzip", compression="gzip", index=True)